======= ======= ======= **DEEP LEARNING E VISAO COMPUTACIONAL** ======= ======= =======
** FACENET - Reconhecimento Facial com opencv-python, mtcnn**
```
Instalar pacotes pelo prompt do conda
pandas
opencv-python
mtcnn
PIL
keras
pickle
numpy
sklearn
```
**Identificar Versões Instaladas**
```
import cv2
print('cv2 Version ',cv2.__version__)
```

** EXEMPLO PARA NOTEBOOK **

** SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder **

In [1]:
#
import os
import subprocess
from cv2 import *
import cv2
from mtcnn.mtcnn import MTCNN

# Classe 
class RegisterImg:
    def __init__(self):
        # Local Arquivo Que Armazenara Dados De Participante
        self.arq_open = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        # Local Gravar Imagem Do Participante
        self.grv_img = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        # Carregar Algoritmo Para 
        self.detector = MTCNN()
        print("= SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =")

    def face_mtcnn_extractor(self, frame):
        """Methods takes in frames from video, extracts and returns faces from them"""
        # Use MTCNN to detect faces in each frame of the video
        result = self.detector.detect_faces(frame)
        return result
    
    def face_localizer(self, person):
        """Method takes the extracted faces and returns the coordinates"""
        # 1. Get the coordinates of the face
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    def gravar_(self):
        os.system('cls')
        print("Gravando...")
        print("Digitar Dados Do Participante")
        matricula = 0
        try:
            matricula = int(input("Número Matrícula -> "))
        except ValueError:
            print("")
            print("== Atenção ==")
            print("Digite Valor Númerico")
            print("              =======")
            menu = MainMenu()
            menu.menu_inicial()

        nome = input("Nome Completo -> ")
        
        # Gravar Imagem A partir Da Camera
        # Inicializar Objeto Que Habilita Camera/WebCam Para Capturar Imagem. 0 = WebCam Notebook/PC
        cam = cv2.VideoCapture(0)
        #Inicializar Objeto Para Detectar Uma Face Na Camera

        while True:
            # Detectar Face Em Cada Frame
            # Capture frame-by-frame
            ret, frame = cam.read()
            if not ret:
                break
            # 1. Extract faces from frames
            result = self.face_mtcnn_extractor(frame)
            #
            k = cv2.waitKey(1)

            # Desenhar Retângulo Na Face Detectada Utilizando As Coordenadas
            for person in result:
                # 2. Localize the face in the frame
                x1, y1, x2, y2, width, height = self.face_localizer(person)

                # 5. Draw a frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 155, 255), 2)

            # display the frame with label
            cv2.imshow("Coletar Imagens - ENTER = Gravar / ESC = Finalizar", frame)

            if k%256 == 27:
                # Se pressionar o ESC
                k = cv2.waitKey(2)
                print("")
                print("ESC pressionado. Fechando Câmera")
                break
            elif k%256 == 13: # Se pressionar o ENTER
                # Converte a imagem para escala de cinza
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Criar Pasta Com Nome do Participante
                os.mkdir(self.grv_img+"train\\"+nome+"_"+str(matricula))
                os.mkdir(self.grv_img+"test\\"+nome+"_"+str(matricula))
                # Persiste Imagem Em Disco
                # Gravara Numero De Matrícula, Sequência Quantidade da Imagem Gravada
                cv2.imwrite(self.grv_img+"train\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])
                cv2.imwrite(self.grv_img+"test\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])

                print("Imagem gravada!")

        # When everything's done, release capture
        cam.release()
        cv2.destroyAllWindows()

        return

Using TensorFlow backend.


In [2]:
#
import datetime
import time
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import savez_compressed, asarray, load, expand_dims
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle

class FaceTrainer:

    def __init__(self):
        self.datasetpath = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        self.faces_npz = "C:\\zprojeto-course\\deeplearn\\dataset\\faces_dataset.npz"
        self.keras_facenet = "C:\\zprojeto-course\\deeplearn\\facenet\\facenet_keras.h5"
        self.faces_embeddings = "C:\\zprojeto-course\\deeplearn\\facerecognition\\faces_dataset_embeddings.npz"
        self.svm_classifier = "C:\\zprojeto-course\\deeplearn\\facerecognition\\SVM_classifier.sav"
        return

    def load_dataset(self, directory):
        """Load a dataset that contains one subdir for each class that in turn contains images"""
        X = []
        y = []
        # enumerate all folders named with class labels
        for subdir in listdir(directory):
            path = directory + subdir + '\\'
            # skip any files that might be in the dir
            if not isdir(path):
                continue
            # load all faces in the subdirectory
            faces = self.load_faces(path)
            # create labels
            labels = [subdir for _ in range(len(faces))]
            print("loaded {} examples for class: {}".format(len(faces), subdir))
            X.extend(faces)
            y.extend(labels)
        return asarray(X), asarray(y)

    def load_faces(self, directory):
        """Load images and extract faces for all images in a directory"""
        faces = []
        # enumerate files
        for filename in listdir(directory):
            path = directory + filename
            # get face
            face = self.extract_face(path)
            faces.append(face)
        return faces

    def extract_face(self, filename, required_size=(160, 160)):
        """Extract a single face from a given photograph"""
        # load image from file
        image = Image.open(filename)
        # convert to RGB, if needed
        image = image.convert('RGB')
        # convert to array
        pixels = asarray(image)
        # create the detector, using default weights
        detector = MTCNN()
        # detect faces in the image
        results = detector.detect_faces(pixels)
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array

    def create_faces_npz(self):
        """Method Creates npz file for all the faces in train_dir, val_dir"""
        # Load the training data set
        trainX, trainy = self.load_dataset(self.datasetpath+"\\train\\")
        print("Training data set loaded")
        # load test dataset
        testX, testy = self.load_dataset(self.datasetpath+"\\test\\")
        print("Testing data set loaded")
        # save arrays to one file in compressed format
        savez_compressed(self.faces_npz, trainX, trainy, testX, testy)
        return

    def create_faces_embedding_npz(self):
        """Create npz file for all the face embeddings in train_dir, val_dir"""
        data = load(self.faces_npz)
        trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
        print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
        # load the facenet model
        model = load_model(self.keras_facenet)
        print('Keras Facenet Model Loaded')
        # convert each face in the train set to an embedding
        newTrainX = list()
        for face_pixels in trainX:
            embedding = self.get_embedding(model, face_pixels)
            newTrainX.append(embedding)
        newTrainX = asarray(newTrainX)
        # convert each face in the test set to an embedding
        newTestX = list()
        for face_pixels in testX:
            embedding = self.get_embedding(model, face_pixels)
            newTestX.append(embedding)
        newTestX = asarray(newTestX)
        # save arrays to one file in compressed format
        savez_compressed(self.faces_embeddings, newTrainX, trainy, newTestX, testy)
        return

    def get_embedding(self, model, face_pixels):
        """Calculate a face embedding for each face in the dataset using facenet
           Get the face embedding for one face"""
        # scale pixel values
        face_pixels = face_pixels.astype('float32')
        # standardize pixel values across channels (global)
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        # transform face into one sample
        samples = expand_dims(face_pixels, axis=0)
        # make prediction to get embedding
        yhat = model.predict(samples)
        return yhat[0]

    def classifier(self):
        """Create a Classifier for the Faces Dataset"""
        # load dataset
        data = load(self.faces_embeddings)
        trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
        print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
        # normalize input vectors
        in_encoder = Normalizer(norm='l2')
        trainX = in_encoder.transform(trainX)
        testX = in_encoder.transform(testX)
        # label encode targets
        out_encoder = LabelEncoder()
        out_encoder.fit(trainy)
        trainy = out_encoder.transform(trainy)
        testy = out_encoder.transform(testy)
        # fit model
        model = SVC(kernel='linear', probability=True)
        model.fit(trainX, trainy)
        # save the model to disk
        filename = self.svm_classifier
        pickle.dump(model, open(filename, 'wb'))
        # predict
        yhat_train = model.predict(trainX)
        yhat_test = model.predict(testX)
        # score
        score_train = accuracy_score(trainy, yhat_train)
        score_test = accuracy_score(testy, yhat_test)
        # summarize
        print('Accuracy: train=%.3f, test=%.3f' % (score_train * 100, score_test * 100))
        return

    def start(self):
        """Method begins the training process"""
        start_time = time.time()
        st = datetime.datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')
        print("----------------------------------------------------------------------------------------")
        print("Face trainer Initiated at {}".format(st))
        print("----------------------------------------------------------------------------------------")
        # Get faces from the images
        self.create_faces_npz()
        # Get embeddings for all the extracted faces
        self.create_faces_embedding_npz()
        # Classify the faces
        self.classifier()
        end_time = time.time()
        et = datetime.datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')
        print("----------------------------------------------------------------------------------------")
        print("Face trainer Completed at {}".format(et))
        print("Total time Elapsed {} secs".format(round(end_time - start_time), 0))
        print("----------------------------------------------------------------------------------------")

        return

In [3]:
import cv2
from PIL import Image
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
import pickle

class FaceDetector:
    
    def __init__(self):
        self.facenet_model = load_model("C:\\zprojeto-course\\deeplearn\\facenet\\facenet_keras.h5")
        self.svm_model = pickle.load(open("C:\\zprojeto-course\\deeplearn\\facerecognition\\SVM_classifier.sav", 'rb'))
        self.data = np.load('C:\\zprojeto-course\\deeplearn\\facerecognition\\faces_dataset_embeddings.npz')
        # object to the MTCNN detector class
        self.detector = MTCNN()

    def face_mtcnn_extractor(self, frame):
        """Methods takes in frames from video, extracts and returns faces from them"""
        # Use MTCNN to detect faces in each frame of the video
        result = self.detector.detect_faces(frame)
        return result

    def face_localizer(self, person):
        """Method takes the extracted faces and returns the coordinates"""
        # 1. Get the coordinates of the face
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    def face_preprocessor(self, frame, x1, y1, x2, y2, required_size=(160, 160)):
        """Method takes in frame, face coordinates and returns preprocessed image"""
        # 1. extract the face pixels
        face = frame[y1:y2, x1:x2]
        # 2. resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        # 3. scale pixel values
        face_pixels = face_array.astype('float32')
        # 4. standardize pixel values across channels (global)
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        # 5. transform face into one sample
        samples = np.expand_dims(face_pixels, axis=0)
        # 6. get face embedding
        yhat = self.facenet_model.predict(samples)
        face_embedded = yhat[0]
        # 7. normalize input vectors
        in_encoder = Normalizer(norm='l2')
        X = in_encoder.transform(face_embedded.reshape(1, -1))
        return X

    def face_svm_classifier(self, X):
        """Methods takes in preprocessed images ,classifies and returns predicted Class label and probability"""
        # predict
        yhat = self.svm_model.predict(X)
        label = yhat[0]
        yhat_prob = self.svm_model.predict_proba(X)
        probability = round(yhat_prob[0][label], 2)
        trainy = self.data['arr_1']
        # predicted label decoder
        out_encoder = LabelEncoder()
        out_encoder.fit(trainy)
        predicted_class_label = out_encoder.inverse_transform(yhat)
        label = predicted_class_label[0]
        return label, str(probability)

    def face_detector(self):
        """Method classifies faces on live cam feed
           Class labels : sai_ram, donald_trump,narendra_modi, virat_koli"""
        # open cv for live cam feed
        cap = cv2.VideoCapture(0)
        while True:
            # Capture frame-by-frame
            __, frame = cap.read()
            # 1. Extract faces from frames
            result = self.face_mtcnn_extractor(frame)
            if result:
                for person in result:
                    # 2. Localize the face in the frame
                    x1, y1, x2, y2, width, height = self.face_localizer(person)
                    # 3. Proprocess the images for prediction
                    X = self.face_preprocessor(frame, x1, y1, x2, y2, required_size=(160, 160))
                    # 4. Predict class label and its probability
                    label, probability = self.face_svm_classifier(X)
                    print(" Person : {} , Probability : {}".format(label, probability))
                    # 5. Draw a frame
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 155, 255), 2)
                    # 6. Add the detected class label to the frame
                    cv2.putText(frame, label+probability, (x1, height),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255),
                                lineType=cv2.LINE_AA)
            #
            k = cv2.waitKey(1)
            # display the frame with label
            cv2.imshow("Identificando Imagem - ESC = Finalizar", frame)
            # break on keybord interuption with 'q'
            if k%256 == 27:
                # Se pressionar o ESC
                k = cv2.waitKey(2)
                print("")
                print("ESC pressionado. Fechando Câmera")
                break

        # When everything's done, release capture
        cap.release()
        cv2.destroyAllWindows()

In [4]:
# Bibliotecas Para 
import os
import subprocess

import cv2
from mtcnn.mtcnn import MTCNN

# Classe 
class MainMenu:
    def __init__(self):
        # Manipular Arquivo Que Armazenara Dados De Participante
        self.arq_open = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        # Nome Do Arquivo que armazenara Dados De Participante
        self.arq_nome = "labelmap.csv"
        # Gravar Imagem Do Participante
        self.grv_img = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        self.detector = MTCNN()
        print("= SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =")

    def menu_inicial(self):
        print("")
        print("Digite O Número Relacionado a Opção Desejada")
        print("")
        print("1 Gravar Imagem")
        print("2 Analisar Imagem")
        print("3 Identificar Imagem")
        print("0 Sair Do Sistema")
        choice = 0
        try:
            choice = int(input(" >> "))
        except ValueError:
            print("")
            print("== Atenção ==")
            print("Digite Valor Númerico")
            print("             =======")
            self.menu_inicial()

        self.choice = choice
        self.chama_acoes(str(self.choice))

    def face_mtcnn_extractor(self, frame):
        """Methods takes in frames from video, extracts and returns faces from them"""
        # Use MTCNN to detect faces in each frame of the video
        result = self.detector.detect_faces(frame)
        return result
    
    def face_localizer(self, person):
        """Method takes the extracted faces and returns the coordinates"""
        # 1. Get the coordinates of the face
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    def gravar_(self):
        os.system('cls')
        print("Gravando...")
        print("Digitar Dados Do Participante")
        gravar = RegisterImg()
        gravar.gravar_()

        os.system('cls')
        self.menu_inicial()
        return

    def analisar_(self):
        print("Analisando Imagem ...")
        analisar = FaceTrainer()
        analisar.start()
        os.system('cls')
        self.menu_inicial()
        return

    def identificar_(self):
        print("Identificando Imagem ...")
        identificar = FaceDetector()
        identificar.face_detector()
        os.system('cls')
        self.menu_inicial()
        return

    def sair_(self):
        print("Sistema Encerrado !")
        os.system('cls')
        quit()
        return

    def chama_acoes(self,ch):
        self.menu_acoes(ch)
        
        return

    def menu_acoes(self,opcao):
        menu_actions = {
                        'main_menu': self.menu_inicial,
                        '1': self.gravar_,
                        '2': self.analisar_,
                        '3': self.identificar_,
                        '0': self.sair_
                        }
        menu_actions[opcao]()
        return

In [5]:
# Chama Classe MainMenu Para Exibir Menu Que Será Manipulado
if __name__ == "__main__":
    os.system('cls')
    menu = MainMenu()
    menu.menu_inicial()

= SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =

Digite O Número Relacionado a Opção Desejada

1 Gravar Imagem
2 Analisar Imagem
3 Identificar Imagem
0 Sair Do Sistema
 >> 1
Gravando...
Digitar Dados Do Participante
= SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =
Gravando...
Digitar Dados Do Participante
Número Matrícula -> 1
Nome Completo -> Carlos Leite
Imagem gravada!

ESC pressionado. Fechando Câmera

Digite O Número Relacionado a Opção Desejada

1 Gravar Imagem
2 Analisar Imagem
3 Identificar Imagem
0 Sair Do Sistema
 >> 2
Analisando Imagem ...
----------------------------------------------------------------------------------------
Face trainer Initiated at 2020-07-02 23:35:40
----------------------------------------------------------------------------------------
loaded 1 examples for class: Aaron_Peirsol
loaded 1 examples for class: Aaron_Sorkin
loaded 1 examples for class: Abdel_Nasser_Assidi
loaded 1 examples 

C:\zprogramas\jupyter\Miniconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Keras Facenet Model Loaded
Dataset: train=29, test=63
Accuracy: train=100.000, test=95.238
----------------------------------------------------------------------------------------
Face trainer Completed at 2020-07-02 23:38:12
Total time Elapsed 152 secs
----------------------------------------------------------------------------------------

Digite O Número Relacionado a Opção Desejada

1 Gravar Imagem
2 Analisar Imagem
3 Identificar Imagem
0 Sair Do Sistema
 >> 3
Identificando Imagem ...


C:\zprogramas\jupyter\Miniconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probability : 0.02
 Person : Carlos Leite_1 , Probabi

In [ ]:
import subprocess
from cv2 import *
import cv2
from mtcnn.mtcnn import MTCNN

# Classe 
class RegisterImg:
    def __init__(self):
        # Local Arquivo Que Armazenara Dados De Participante
        self.arq_open = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        # Local Gravar Imagem Do Participante
        self.grv_img = "C:\\zprojeto-course\\deeplearn\\dataset\\"
        # Carregar Algoritmo Para 
        self.detector = MTCNN()
        print("= SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =")

    def face_mtcnn_extractor(self, frame):
        """Methods takes in frames from video, extracts and returns faces from them"""
        # Use MTCNN to detect faces in each frame of the video
        result = self.detector.detect_faces(frame)
        return result
    
    def face_localizer(self, person):
        """Method takes the extracted faces and returns the coordinates"""
        # 1. Get the coordinates of the face
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    def gravar_(self):
        os.system('cls')
        print("Gravando...")
        print("Digitar Dados Do Participante")
        matricula = 0
        try:
            matricula = int(input("Número Matrícula -> "))
        except ValueError:
            print("")
            print("== Atenção ==")
            print("Digite Valor Númerico")
            print("              =======")
            menu = MainMenu()
            menu.menu_inicial()

        nome = input("Nome Completo -> ")
        
        # Gravar Imagem A partir Da Camera
        # Inicializar Objeto Que Habilita Camera/WebCam Para Capturar Imagem. 0 = WebCam Notebook/PC
        cam = cv2.VideoCapture(0)
        #Inicializar Objeto Para Detectar Uma Face Na Camera

        while True:
            # Detectar Face Em Cada Frame
            # Capture frame-by-frame
            ret, frame = cam.read()
            if not ret:
                break
            # 1. Extract faces from frames
            result = self.face_mtcnn_extractor(frame)
            #
            k = cv2.waitKey(1)

            # Desenhar Retângulo Na Face Detectada Utilizando As Coordenadas
            for person in result:
                # 2. Localize the face in the frame
                x1, y1, x2, y2, width, height = self.face_localizer(person)

                # 5. Draw a frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 155, 255), 2)

            # display the frame with label
            cv2.imshow("Coletar Imagens - ENTER = Gravar / ESC = Finalizar", frame)

            if k%256 == 27:
                # Se pressionar o ESC
                k = cv2.waitKey(2)
                print("")
                print("ESC pressionado. Fechando Câmera")
                break
            elif k%256 == 13: # Se pressionar o ENTER
                # Converte a imagem para escala de cinza
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Criar Pasta Com Nome do Participante
                os.mkdir(self.grv_img+"train\\"+nome+"_"+str(matricula))
                os.mkdir(self.grv_img+"test\\"+nome+"_"+str(matricula))
                # Persiste Imagem Em Disco
                # Gravara Numero De Matrícula, Sequência Quantidade da Imagem Gravada
                cv2.imwrite(self.grv_img+"train\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])
                cv2.imwrite(self.grv_img+"test\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])

                print("Imagem gravada!")

        # When everything's done, release capture
        cam.release()
        cv2.destroyAllWindows()

        return